<a href="https://colab.research.google.com/github/miladqolamee/NeuroMatch_2022_group_project/blob/main/Coding_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook is created for exploratory data analysis on Allen 2P dataset by **Pierogi All Stras** 

---
# A brief summary of Allen 2P Experiment Design


- Single and multiple-plane 2 photon calcium imaging was used.
- Exitatory and inhibitory populaiton of nuerons were imaged in different cre-lines.
- Each population was imaged over multiple days and under different experimental conditions (with familiar, novel stimuli and under active and passive veiwing conditions)
- This dataset can be used to evaluate the influence of experience, expectation and task engagemen on neural coding and dynamics.
- Overal, we have 82 mouse, 3021 behavioral sessions and 551 imaging sessions. In total 34619 cells were recorded.
- Mice performed a go/no go visual change detection task. Visual stimui were presented continuously for 1 hour. The start of trial was not indicated and mouse was free to run.

For more information see:
[Link](https://allensdk.readthedocs.io/en/latest/visual_behavior_optical_physiology.html).

# Experimental setup 

<img src="https://github.com/miladqolamee/NeuroMatch_2022_group_project/blob/main/Allen%20viewing%20condition.png?raw=1" alt="Experimental setup" class="bg-primary" width="75%">

---
# A brief summary of Allen 2P Dataset
To access data in the dataset we can instal `allensdk` package. By using `AllenBehaviorOphysProjectCache` object and its methods in this package we can access desired data. 

In the dataset we have 3 tables and 2 datasets streams. We should use informatin inside the table to extract our desired subset of information. These 3 tables are:

1) Behavior Sessions Table: Contains metadata for behavioral sessions (both trianing session and also sessions whcih imaging was perfomed). This table can be accessed by `get_behavior_session_talbe()` method.

2) Ophys Sessions Table: Contains metadata and experimental conditions for all sessions. This talbe can accessed by `get_ophys_session_table()` method.

3) Ophys Experiments Table: Contains metadata and experimental conditions fo all imaging experiment (aka imaging plane). This table can be accessed by `get_experiment_session_table()` method.

For more information see:
[Link](https://github.com/miladqolamee/NeuroMatch_2022_group_project/blob/main/VBP_WhitePaper_SDK_Documentation.pdf).

# Allen 2P dataset terminology

<img src="https://allensdk.readthedocs.io/en/latest/_static/visual_behavior_2p/data_structure.png" alt="dataset terminology" class="bg-primary" width="100%">

---
# First instal and upgrade allensdk package

In [ ]:
# @title Instal allensdk package
# @markdown Run this cell and then restart the Runtime.

# @markdown After running this cell for the first time you don't nee to rerun it.
#pip install allensdk
!pip install allensdk
!pip install hdmf --upgrade

In [ ]:
#  @title Upgrade the package
!pip install --upgrade pip
!pip install allensdk


In [3]:
#  @title Improt relevant packages
import numpy as np
import pandas as pd
from pathlib import Path
import math

import allensdk
from allensdk.brain_observatory.behavior.behavior_project_cache import VisualBehaviorOphysProjectCache

## First we need a file path because we need to pass it to cache object to indicate where the file should be downloaded

## By calling `VisualBehaviorOphysProjectCache`, Tables will be downloaded

In [4]:
# Update this to a valid directory in your filesystem
data_storage_directory = Path("/local1/visual_behavior_ophys_cache_dir")

cache = VisualBehaviorOphysProjectCache.from_s3_cache(cache_dir=data_storage_directory)

ophys_session_table.csv: 100%|██████████| 227k/227k [00:00<00:00, 1.02MMB/s] 
behavior_session_table.csv: 100%|██████████| 1.21M/1.21M [00:00<00:00, 5.48MMB/s]
ophys_experiment_table.csv: 100%|██████████| 610k/610k [00:00<00:00, 2.91MMB/s] 
ophys_cells_table.csv: 100%|██████████| 4.29M/4.29M [00:00<00:00, 8.46MMB/s]


## Experiment Sessions Table

In [ ]:
experiment_table = cache.get_ophys_experiment_table()   # Getting Experiment Session Table
experiment_table.sample(5)                              # Show 5 random observation
print(len(experiment_table))                            # How many rows?
print(len(experiment_table.ophys_session_id.unique()))  # How many unique imaging session

## Selecting experiments for VIP and SST nuerons

In [30]:
experiment_variables = experiment_table.columns      # All variables in behavior table

experiment_table.cre_line.unique()    # How many unique cre-line we have

Vip_experiment_table = experiment_table[(experiment_table.cre_line.values == 'Vip-IRES-Cre') & 
                                    (experiment_table.ophys_session_id.isnull() == False)]
Sst_experiment_table = experiment_table[(experiment_table.cre_line.values == 'Sst-IRES-Cre') & 
                                    (experiment_table.ophys_session_id.isnull() == False)]


# getting information just for one mouse for simplification and sort its table by date of aquisition
Vip_mouse_id = Vip_experiment_table.mouse_id.unique()[0]
Sst_mouse_id = Sst_experiment_table.mouse_id.unique()[0]

this_mouse_Vip_experiment_table = experiment_table[(experiment_table.cre_line.values == 'Vip-IRES-Cre') & 
                                    (experiment_table.ophys_session_id.isnull() == False)
                                    & (experiment_table.mouse_id.values == Vip_mouse_id)].sort_values(
                                        by = 'date_of_acquisition')
this_mouse_Sst_experiment_table = experiment_table[(experiment_table.cre_line.values == 'Sst-IRES-Cre') & 
                                    (experiment_table.ophys_session_id.isnull() == False)
                                    & (experiment_table.mouse_id.values == Sst_mouse_id)].sort_values(
                                        by = 'date_of_acquisition')



,equipment_name,full_genotype,mouse_id,reporter_line,driver_line,sex,age_in_days,cre_line,indicator,session_number,...,ophys_container_id,project_code,imaging_depth,targeted_structure,date_of_acquisition,session_type,experience_level,passive,image_set,file_id
ophys_experiment_id,,,,,,,,,,,,,,,,,,,,,
792815735,CAM2P.4,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,412036,Ai148(TIT2L-GC6f-ICL-tTA2),[Vip-IRES-Cre],F,148.0,Vip-IRES-Cre,GCaMP6f,1.0,...,791352433,VisualBehavior,175,VISp,2018-12-10 21:19:50.000000,OPHYS_1_images_A,Familiar,False,A,859887405
795073741,CAM2P.4,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,412036,Ai148(TIT2L-GC6f-ICL-tTA2),[Vip-IRES-Cre],F,151.0,Vip-IRES-Cre,GCaMP6f,3.0,...,791352433,VisualBehavior,175,VISp,2018-12-13 21:11:17.000000,OPHYS_3_images_A,Familiar,False,A,859887459
795953296,CAM2P.4,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,412036,Ai148(TIT2L-GC6f-ICL-tTA2),[Vip-IRES-Cre],F,152.0,Vip-IRES-Cre,GCaMP6f,4.0,...,791352433,VisualBehavior,175,VISp,2018-12-14 20:17:05.000000,OPHYS_4_images_B,Novel 1,False,B,859887431
796108483,CAM2P.4,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,412036,Ai148(TIT2L-GC6f-ICL-tTA2),[Vip-IRES-Cre],F,153.0,Vip-IRES-Cre,GCaMP6f,5.0,...,791352433,VisualBehavior,175,VISp,2018-12-15 20:56:37.000000,OPHYS_5_images_B_passive,Novel >1,True,B,859888310
796308505,CAM2P.4,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,412036,Ai148(TIT2L-GC6f-ICL-tTA2),[Vip-IRES-Cre],F,154.0,Vip-IRES-Cre,GCaMP6f,6.0,...,791352433,VisualBehavior,175,VISp,2018-12-16 20:44:48.000000,OPHYS_6_images_B,Novel >1,False,B,859887448
798404219,CAM2P.4,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,412036,Ai148(TIT2L-GC6f-ICL-tTA2),[Vip-IRES-Cre],F,156.0,Vip-IRES-Cre,GCaMP6f,2.0,...,791352433,VisualBehavior,175,VISp,2018-12-18 23:26:29.000000,OPHYS_2_images_A_passive,Familiar,True,A,859888258


For this specific Vip mouse we have six imaging session sorted by date of aquisition. In `session_type` column it can be seen what type of image was presented and what kind of task (active or passive ) was performed. 
In the next cell we want to compare the neural response around stimulus omittion during active and passive task.

In [31]:
# getting passive and active experiment ids for this mouse (just one active and one passive session where "image_B" was presented)
this_mouse_Vip_passive_experiment_id = this_mouse_Vip_experiment_table[this_mouse_Vip_experiment_table.session_type.str.contains("passive")].index[0]
this_mouse_Vip_active_experiment_id = this_mouse_Vip_experiment_table[~this_mouse_Vip_experiment_table.session_type.str.contains("passive")].index[2]


# loading experiment dataset 
this_mouse_Vip_passive_experiment = cache.get_behavior_ophys_experiment(this_mouse_Vip_passive_experiment_id)
this_mouse_Vip_active_experiment = cache.get_behavior_ophys_experiment (this_mouse_Vip_active_experiment_id)

Look at the performance of these two sessions (one active and the other passive)  
* You can run each line or varialble of the code by highlighting it and then press `control` + `shift`+ `inter`

In [38]:
this_mouse_Vip_passive_experiment.get_performance_metrics()   # perfomance metrics of this passice session
this_mouse_Vip_active_experiment.get_performance_metrics()    # perfomance metrics of this active session


{'auto_reward_count': 0,
 'catch_trial_count': 51,
 'correct_reject_trial_count': 51,
 'earned_reward_count': 0,
 'engaged_trial_count': 0,
 'false_alarm_trial_count': 0,
 'go_trial_count': 351,
 'hit_trial_count': 0,
 'max_dprime': 0.0,
 'max_dprime_engaged': nan,
 'maximum_reward_rate': 0.0,
 'mean_dprime': 0.0,
 'mean_dprime_engaged': nan,
 'mean_false_alarm_rate': 0.057230044349333695,
 'mean_false_alarm_rate_engaged': nan,
 'mean_false_alarm_rate_uncorrected': 0.0,
 'mean_hit_rate': 0.011195078480863365,
 'mean_hit_rate_engaged': nan,
 'mean_hit_rate_uncorrected': 0.0,
 'miss_trial_count': 351,
 'total_reward_count': 0,
 'total_reward_volume': 0.0,
 'trial_count': 402}

In [ ]:
#Just a code that might help later on
#M='M'
#behavior_table.query('sex == @ M')
#This way we can specifically ask for the data containing these parameters. the mouse only being male (which is better since female mices' results aren't as reliable)